In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import sys
#sys.path.append('/content/drive/MyDrive/notebook_data')
#!pip install -r /content/drive/MyDrive/notebook_data/requirements.txt
import numpy as np
import pandas as pd
#from preprocess import preprocess_dataframe, bsk_preprocessor
import joblib

In [3]:
#df = pd.read_csv('/content/drive/MyDrive/notebook_data/datasets/merged-labeled-reduced-2.tsv',sep='\t') # assuming column called ['text']
# Delete multiple columns
#df.drop(['id', 'author', 'handle', 'timestamp','query'], axis='columns', inplace=True)
#df.head()
df = pd.read_csv('/content/drive/MyDrive/notebook_data/datasets/merged-labeled-reduced-2-preprocessed-v2.tsv',sep='\t')
df.head()

,text,label,cleaned
0,\U0001f6a8 Severe Thunderstorm Warning issued ...,flood,police car light severe thunderstorm warning i...
1,\U0001f6a8 Severe Thunderstorm Warning issued ...,flood,police car light severe thunderstorm warning i...
2,\U0001f6a8 Severe Thunderstorm Warning issued ...,flood,police car light severe thunderstorm warning i...
3,\U0001f6a8 Severe Thunderstorm Warning issued ...,flood,police car light severe thunderstorm warning i...
4,NEW WEATHER ADVISORY: Severe Thunderstorm Warn...,flood,new weather advisory severe thunderstorm warni...


In [4]:
df = df.dropna()
df['label'].value_counts()

,count
label,
other,9937
hurricane,4746
wildfire,4084
flood,3472
blizzard,2535
tornado,520


In [ ]:
#df = preprocess_dataframe(df)
#df.dropna()
#df.head()
#df.to_csv('/content/drive/MyDrive/notebook_data/datasets/merged-labeled-reduced-2-preprocessed-v2.tsv',sep='\t',index=False)

In [5]:
#!pip install xgboost
#!pip install lightgbm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import StratifiedKFold, HalvingGridSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix

#df.head()

In [6]:
# train test split
X = df['cleaned']
y = df['label']

from sklearn.preprocessing import LabelEncoder

# Encode string labels into integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Converts labels to numeric form
print(np.unique(y))

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)
print(f"Training with {len(X_train)} samples; Testing with {len(X_test)} samples")
print(np.unique(y_train))

['blizzard' 'flood' 'hurricane' 'other' 'tornado' 'wildfire']
Training with 20235 samples; Testing with 5059 samples
[0 1 2 3 4 5]


In [7]:
# define pipeline for models
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', None) # placeholder
])

In [8]:
# define parameter grids
tfidf_params = {
    'tfidf__max_df': [0.8],
    'tfidf__min_df': [2, 3],
    'tfidf__ngram_range': [(1,1)],
    #'tfidf__use_idf': [True, False],
    #'tfidf__smooth_idf': [True, False]
}

lgbm_param_grid = {
    'clf': [LGBMClassifier()],
    'clf__learning_rate': [0.01, 0.05],
    'clf__n_estimators': [150, 200, 250],
    'clf__num_leaves': [15, 20],
    'clf__min_child_samples': [15, 20],
    'clf__colsample_bytree': [0.7],
    #'clf__min_child_weight': [0.2, 0.3],
    'clf__reg_alpha': [1.5, 2.0],
    'clf__reg_lambda': [1.5, 2.0],
    #'clf__subsample': [0.6, 0.7],
    'clf__force_row_wise': [True],
    #'clf__is_unbalance': [True],
    **tfidf_params
}

In [9]:
# set up the grid search
#param_grid = [logreg_param_grid, linsvc_param_grid, svc_param_grid, xgb_param_grid, lgbm_param_grid]
param_grid = [lgbm_param_grid]
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = HalvingGridSearchCV(pipeline, param_grid, cv=kfolds, n_jobs=-1, verbose=1, scoring='f1', factor=3)

In [10]:
# fit to the data
grid_search.fit(X_train, y_train)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 249
max_resources_: 20235
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 192
n_resources: 249
Fitting 5 folds for each of 192 candidates, totalling 960 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/model_selectio

----------
iter: 1
n_candidates: 64
n_resources: 747
Fitting 5 folds for each of 64 candidates, totalling 320 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan

----------
iter: 2
n_candidates: 22
n_resources: 2241
Fitting 5 folds for each of 22 candidates, totalling 110 fits


...

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan

----------
iter: 4
n_candidates: 3
n_resources: 20169
Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan

[LightGBM] [Info] Total Bins 106041
[LightGBM] [Info] Number of data points in the train set: 20235, number of used features: 3213
[LightGBM] [Info] Start training from score -2.300364
[LightGBM] [Info] Start training from score -1.985683
[LightGBM] [Info] Start training from score -1.673202
[LightGBM] [Info] Start training from score -0.934368
[LightGBM] [Info] Start training from score -3.884484
[LightGBM] [Info] Start training from score -1.823542


HalvingGridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                    estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                              ('clf', None)]),
                    n_jobs=-1,
                    param_grid=[{'clf': [LGBMClassifier()],
                                 'clf__colsample_bytree': [0.7],
                                 'clf__force_row_wise': [True],
                                 'clf__learning_rate': [0.01, 0.05],
                                 'clf__min_child_samples': [15, 20],
                                 'clf__n_estimators': [150, 200, 250],
                                 'clf__num_leaves': [15, 20],
                                 'clf__reg_alpha': [1.5, 2.0],
                                 'clf__reg_lambda': [1.5, 2.0],
                                 'tfidf__max_df': [0.8],
                                 'tfidf__min_df': [2, 3],
                                 'tfidf__ngram_range': [(1, 1)]}],
                    scoring='f1', verbose=1)

In [49]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'clf': LGBMClassifier(), 'clf__colsample_bytree': 0.7, 'clf__force_row_wise': True, 'clf__is_unbalance': True, 'clf__learning_rate': 0.01, 'clf__min_child_samples': 15, 'clf__min_child_weight': 0.2, 'clf__n_estimators': 250, 'clf__num_leaves': 20, 'clf__reg_alpha': 2.0, 'clf__reg_lambda': 2.5, 'clf__subsample': 0.7, 'tfidf__max_df': 0.75, 'tfidf__min_df': 0.005, 'tfidf__ngram_range': (1, 1)}
Best score: nan


In [50]:
best_estimator = grid_search.best_estimator_
y_pred = best_estimator.predict(X_test)
# convert them back to the labels we can understand
y_pred_text = label_encoder.inverse_transform(y_pred)
y_test_text = label_encoder.inverse_transform(y_test)
print(classification_report(y_test_text, y_pred_text), confusion_matrix(y_test_text, y_pred_text))

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


              precision    recall  f1-score   support

    blizzard       0.81      0.85      0.83       507
       flood       0.89      0.87      0.88       694
   hurricane       0.95      0.93      0.94       949
       other       0.83      0.86      0.84      1988
     tornado       0.89      0.79      0.84       104
    wildfire       0.84      0.80      0.82       817

    accuracy                           0.86      5059
   macro avg       0.87      0.85      0.86      5059
weighted avg       0.86      0.86      0.86      5059
 [[ 432    3    2   70    0    0]
 [   4  605   13   58    4   10]
 [   0   25  878   42    2    2]
 [  99   41   28 1702    4  114]
 [   0    3    1   18   82    0]
 [   0    2    2  157    0  656]]


In [39]:
cv_results = grid_search.cv_results_

# Print mean training and validation scores for each parameter set
for mean_train, mean_val, params in zip(cv_results["mean_train_score"], cv_results["mean_test_score"], cv_results["params"]):
    print(f"Params: {params}")
    print(f"Train Score: {mean_train:.4f} | Validation Score: {mean_val:.4f}\n")

print("Best Validation Score:", grid_search.best_score_)

Streaming output truncated to the last 5000 lines.

Params: {'clf': LGBMClassifier(), 'clf__colsample_bytree': 0.8, 'clf__force_row_wise': True, 'clf__is_unbalance': True, 'clf__learning_rate': 0.05, 'clf__min_child_samples': 15, 'clf__min_child_weight': 0.2, 'clf__n_estimators': 120, 'clf__num_leaves': 20, 'clf__reg_alpha': 1.2, 'clf__reg_lambda': 2.0, 'clf__subsample': 0.7, 'tfidf__max_df': 0.8, 'tfidf__min_df': 0.01, 'tfidf__ngram_range': (1, 1)}
Train Score: 0.2437 | Validation Score: 0.2667

Params: {'clf': LGBMClassifier(), 'clf__colsample_bytree': 0.8, 'clf__force_row_wise': True, 'clf__is_unbalance': True, 'clf__learning_rate': 0.05, 'clf__min_child_samples': 15, 'clf__min_child_weight': 0.2, 'clf__n_estimators': 120, 'clf__num_leaves': 20, 'clf__reg_alpha': 1.2, 'clf__reg_lambda': 2.0, 'clf__subsample': 0.7, 'tfidf__max_df': 0.85, 'tfidf__min_df': 0.001, 'tfidf__ngram_range': (1, 1)}
Train Score: 0.2437 | Validation Score: 0.2667

Params: {'clf': LGBMClassifier(), 'clf__colsam

In [92]:
# export model
joblib.dump((label_encoder, best_estimator), '/content/drive/MyDrive/notebook_data/lgbm_model_v5_encoder.pkl')

['/content/drive/MyDrive/notebook_data/lgbm_model_v5.pkl']